## 1. 图像的读取和显示
[OpenCV-Python快速入门（一）：基本操作](https://blog.csdn.net/FriendshipTang/article/details/125830443)
* 主要函数接口  
cv2.imread("file-path") // 读取图片成为2维或者3维的矩阵

* 显示图像(窗口显示)  
cv2.namedWindow("img",0) // 用来创建指定名称的窗口,0表示CV_WINDOW_NORMAL  
cv2.resizeWindow("img", 640, 480); // 设置长宽大小为640*480  
cv2.imshow("img", img) // 用来显示图像  
// 用来等待按键，当用户按下键盘后，该语句会被执行，并获取返回值  
cv2.waitKey(0) // 值是负数或者零时，表示无限等待  
cv2.destroyAllWindows() // 用来释放（销毁）所有窗口  

* `namedWindow("name", arg)`
  arg = WINDOW_NORMAL, WINDOW_AUTOSIZE, WINDOW_KEEPRATIO, WINDOW_FREERATIO, 


In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [3]:
ninu = cv2.imread("../imagedata/nilu.png")
ninu.shape # RGB三通道

(782, 750, 3)

In [4]:
def img_show(title, img):
    cv2.namedWindow(title, 1) # 可调整宽高
    cv2.imshow(title, img)
    cv2.waitKey(0) # 0或者负数表示无限等待, 正数表示等待N毫秒就关闭窗口
    cv2.destroyAllWindows() # 释放窗口
img_show("ninu", ninu)
# 保存图片
# cv2.imwrite("path", img)

## 视频的保存和播放
主要函数接口
* `VideoCapture(arg)`: arg=0表示读取摄像头, arg="video path"则读取该视频. 
* `cap.isOpened()`: 判断视频对象是否成功读取，成功读取视频对象返回True.  
* `ret,frame = cap.read()`: 按帧读取视频，返回值ret是布尔型，正确读取则返回True，读取失败或读取视频结尾则会返回False, frame为每一帧的图像.  
* cv2.waitKey(1)，waitKey（）方法本身表示等待键盘输入，参数是1，表示延时1ms切换到下一帧图像，对于视频而言；参数为0，如cv2.waitKey(0)只显示当前帧图像，相当于视频暂停, 得到的是键盘输入的ASCII码，esc键对应的ASCII码是27。


### 视频播放
* 基本原理是获取每一帧, 将其按照一定的时延展示出来

In [5]:
def playVideo(path, name, size):
    video = cv2.VideoCapture(path)
    # 获取视频的基本信息
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS) # 帧率, 也即每秒传输的帧数
    fourcc = video.get(cv2.CAP_PROP_FOURCC) # 视频编码格式
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        # 显示帧, 注意有一个窗口放缩操作
        cv2.namedWindow(name, 0)
        if width > size:
            cv2.resizeWindow(name, size, int(size*height/width))
        cv2.imshow(name, frame)
        # 控制每一帧的间隔, 以及用户操作, 例如按一下空格就暂停, 再按一下就播放
        key = cv2.waitKey(int(1000/fps))
        if key == 32: # 表示空格, 应该暂停
            key = cv2.waitKey(0) # 暂停帧, 等待键盘输入
            if key == 32: # 表示再次按下空格, 视频播放, 也即进入下一帧的处理
                continue
            if key == 27: # 按下Esc直接退出
                break
        if key == 27: # 按下Esc直接退出
                break
    video.release()
    cv2.destroyAllWindows()
    return width, height, fps, fourcc
playVideo("../imagedata/JapanOceanSunrise.mp4", "JapanOceanSunrise", 1080)

(3840, 2160, 30.0, 828601953.0)

### 视频流写入与保存
一帧一帧的写入
* `VideoWriter_fourcc`: 保存视频编码格式  
* `VideoWriter`: 写入视频对象句柄  
* 关于fourcc:
    1. .avi
        $I420$ YUV编码, $PIM1$ MPEG-1编码, $XVID$ MPEG-4编码  
    2. .mp4 MP4V


In [49]:
import os
fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', 'V')
outV1 = cv2.VideoWriter("../imagedata/demo.mp4", fourcc, 30, (1920, 1080))
imgRoot = "./input/"
for f in os.listdir(imgRoot):
    frame = cv2.imread(imgRoot+f)
    for j in range(30):
        outV1.write(frame)
outV1.release()
cv2.destroyAllWindows()


## cv绘制线段、射线、直线、矩形、椭圆、四边形

In [2]:
def init_img(h, w, c):
    img = np.ones((h, w, 3), dtype=np.uint8)
    img = img*c
    return img
def show(img):
    cv2.imshow("img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
show(init_img(600, 800, 224))

##### img = cv2.line( img, pt1, pt2, color[, thickness[, lineType ]])

In [36]:
# (w, h)
blue_line = cv2.line(
    init_img(600, 800, 224), 
    (200, 150), 
    (600, 450), 
    (255, 127, 63), 
    2
)
show(blue_line)

##### img = cv2.rectangle( img, pt1, pt2, color[, thickness[, lineType ]])

In [37]:
rec_tan = cv2.rectangle(
    init_img(600, 800, 224), 
    (200, 150), 
    (600, 450), 
    (255, 127, 63), 
    1  # 若为-1则表示实心
)
show(rec_tan)

##### img = cv2.circle( img, center, radius, color[, thickness[, lineType]] )

In [42]:
circle = cv2.circle(
    init_img(600, 800, 224), 
    (400, 300), 
    150, 
    (255, 127, 63), 
    -1  # 若为-1则表示实心
)
show(circle)

##### img=cv2.ellipse(img, center, axes, angle, startAngle, endAngle, color[,thickness[, lineType]])

In [46]:
ellipse = cv2.ellipse(
    init_img(600, 800, 224), 
    (400, 300), 
    (200, 150), # (横向半轴长, 纵向半轴长)
    45, # 偏角0至360
    45, # 始角
    315, # 终角
    (255, 127, 63), 
    2  # 若为-1则表示实心
)
show(ellipse)

##### img = cv2.polylines( img, pts, isClosed, color[, thickness[, lineType[, shift]]])

In [54]:
pts = np.array([[200, 160], [200, 480], [400, 320]])
poly = cv2.polylines(
    init_img(600, 800, 224), 
    [pts],
    True,
    (255, 127, 63), 
    2
)
show(poly)

## 文字显示
```
img=cv2.putText(img, text, org, fontFace, fontScale, color[, thickness[,lineType[, bottomLeftOrigin]]])
参数说明：
    img：在其上面绘制图形的载体图像（绘图的容器载体，也称为画布、画板）。
    text 为要绘制的字体。
    org 为绘制字体的位置，以文字的左下角为起点。
    fontFace 表示字体类型，其参数类型及含义。
    fontScale 表示字体大小。
    color：绘制形状的颜色。通常使用 BGR 模型表示颜色，
        例如，(0, 255, 0)表示绿色。对于灰度图像，只能传入灰度值。
        需要注意，颜色通道的顺序是 BGR，而不是 RGB。
    thickness：线条的粗细。
        默认值是 1，如果设置为-1，表示填充图形（即绘制的图形是实心的）。
    lineType：线条的类型，默认是 8 连接类型。
    bottomLeftOrigin 用于控制文字的方向。
        默认值为 False，当设置为 True 时，文字是垂直镜像的效果。https://blog.csdn.net/FriendshipTang/article/details/126088388
```

In [10]:
img = init_img(600, 800, 224)
img = cv2.putText(
    img, 
    "Hello, Open-CV!", 
    (100, 250), 
    cv2.FONT_HERSHEY_COMPLEX, 
    2,
    (255, 127, 63),
    5
)
# show(img)

关于中文显示

In [11]:
from PIL import Image, ImageDraw, ImageFont
# 绘制中文
img = Image.fromarray(img)
draw = ImageDraw.Draw(img)
# 字体格式
font = ImageFont.truetype("stsong.ttf", size=50, encoding='utf-8')
# 绘制文本
draw.text((100, 400),"数字世界，精彩无限。",(255, 127, 63),font=font)
img = np.array(img)
show(img)

#### 我想要使图片有一个渐变的效果，例如逐渐变淡(变白变亮?)
我的理解是
1. 首先你需要将图片进行放缩和边缘填充以适合视频尺寸
2. 紧接着对图像进行渐进式的像素变化，比如统一变大
值得注意的是，或许已经有了这样的一个api可以调用:
cv2.addWeighted(bottom, ratio, top, 1-ratio, gamma)

In [3]:
# 将img按规定尺寸和底色进行标准化
def stdize(img, w, h, c):
    iw = img.shape[1]
    ih = img.shape[0]
    std = np.ones((h, w, 3), dtype=np.uint8)*c
    # 放缩
    if iw > w: # 宽度超限
        ih = int(w*ih/iw)
        iw = w
        img = cv2.resize(img, (iw, ih)) # 注意为宽, 高
    if ih > h: # 高度超限
        iw = int(h*iw/ih)
        ih = h
        img = cv2.resize(img, (iw, ih))
    # 填充
    sx = int((w-iw)/2)
    sy = int((h-ih)/2)
    std[sy: sy+ih, sx: sx+iw, 0: 3] = img
    return std        

In [4]:
# 现规定图片或者视频帧的统一大小为1024x618, 背景底色224
# 字体颜色(255, 127, 63), 帧率30
DY_WIDTH = 1024
DY_HEIGHT = 618
DY_BGCOLOR = 224
DY_FONTCOL = (255, 127, 63)
FPS = 30
potrait = cv2.imread("../imagedata/dy.jpg")
potrait = stdize(potrait, DY_WIDTH, DY_HEIGHT, DY_BGCOLOR)
show(potrait)

In [8]:
bgcanvas = init_img(DY_HEIGHT, DY_WIDTH, DY_BGCOLOR)
for i in np.linspace(0, 1, 180):
    frame = cv2.addWeighted(bgcanvas, i, potrait, 1-i, 0)
    cv2.imshow("potrait", frame)
    cv2.waitKey(10)
cv2.destroyAllWindows()

In [27]:
def init_video(path, fps, size):
    fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', 'V')
    return cv2.VideoWriter(path, fourcc, fps, size)
path = "22221324.mp4"
vhd = init_video(path, FPS, (DY_WIDTH, DY_HEIGHT))

# 1. 片头, 照片配一段文字，要求先显示照片，效果为淡入, 然后逐个的显示文字，最后慢慢淡出
# 1s是30帧，照片淡入3s，逐个显示文字9s，整体淡出3s, 一共15x30=450帧
# 图片淡入
t_in = 3
wt = int(1000/FPS)
frame = bgcanvas
info = "大家好我是段裕，学号22221324" # 18
for i in np.linspace(0, 1, t_in*FPS):
    frame = cv2.addWeighted(bgcanvas, 1-i, potrait, i, 0)
    vhd.write(frame)
    cv2.imshow("potrait", frame)
    cv2.waitKey(wt)
# 逐个显示文字
w = int((DY_WIDTH-len(info)*30)/2)
h = int(DY_HEIGHT-30*3)
for i in range(len(info)):
    frame = addCh(frame, info[:i], w, h, 30, DY_FONTCOL)
    cv2.imshow("potrait", frame)
    cv2.waitKey(200)
def addCh(img, text, w, h, size, c):
    img = Image.fromarray(img)
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype("stsong.ttf", size=size, encoding='utf-8')
    draw.text((w, h),text, c, font=font)
    img = np.array(img)
    return img

cv2.destroyAllWindows()